In [31]:
import datetime
import json
import os
import botocore
from fake_useragent import UserAgent
import boto3
from scrapy.crawler import CrawlerRunner
from scrapy.utils.project import get_project_settings
from crochet import setup
import threading


# Initialize Crochet
setup()

ua = UserAgent()
# region = os.getenv('ap-northeast-1')
# sqs = boto3.client('sqs', region_name=region, config=botocore.client.Config(max_pool_connections=500))
# queue_tier_links = 'https://sqs.ap-northeast-1.amazonaws.com/407620147666/aws_crawler_tier4_links.fifo'

In [60]:
import hashlib
import json
import os
import threading
from urllib.parse import urlparse

from scrapy_selenium import SeleniumRequest
import botocore
import scrapy
from fake_useragent import UserAgent
import boto3
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.wait import WebDriverWait

ua = UserAgent()
# region = os.getenv('ap-northeast-1')
# sqs = boto3.client('sqs', region_name=region, config=botocore.client.Config(max_pool_connections=500))
# s3 = boto3.client('s3', region_name=region, config=botocore.client.Config(max_pool_connections=500))
export_to_s3 = 'Off'
# queue_tier_content_links = 'https://sqs.ap-northeast-1.amazonaws.com/407620147666/aws_crawler_tier5_content_links.fifo'
# s3_tier_content_links = os.getenv('S3_tier_content_linkS')
# s3_crawler_content_folder = os.getenv('S3_CRAWLER_CONTENT_FOLDER')


def hash_function(original_string):
    # Create a hash object using SHA-256 algorithm
    hasher = hashlib.sha256()

    # Convert the input string to bytes (required by hashlib)
    input_bytes = original_string.encode('utf-8')

    # Update the hash object with the input bytes
    hasher.update(input_bytes)

    # Get the hexadecimal representation of the hash value
    return hasher.hexdigest()



class IOpenMallSpider(scrapy.Spider):
    name = "iopenmall" # modify
    allowed_domains = ["mall.iopenmall.tw/iopen/"]  # modify
    start_urls = "https://mall.iopenmall.tw/iopen/"  # modify
    user_agent = ua.random
    runner = None
    category_links = []

    def __init__(self, runner=None, **kwargs):
        self.runner = runner
        super(IOpenMallSpider, self).__init__(**kwargs) # modify

    def start_requests(self):
        # Start the initial request to fetch category links
        print("Starting start_requests")
        if self.runner.target_url == "":
            target_url = json.loads(self.runner.tier4_category_object['Body'])['category_link']
        else:
            target_url = self.runner.target_url

        headers = {"Host": urlparse(target_url).netloc,
                   'Accept-Encoding': 'gzip, deflate, br',
                   'Accept-Language': 'en-US,en;q=0.9,zh-TW;q=0.8,zh;q=0.7,zh-CN;q=0.6,ja;q=0.5',
                   'Sec-Fetch-Dest': 'document',
                   'Sec-Fetch-Mode': 'navigate',
                   'Sec-Fetch-Site': 'none',
                   'Upgrade-Insecure-Requests': '1',
                   "Referer": "https://www.momoshop.com.tw/main/Main.jsp",
                   "User-Agent": ua.random}
        print(f'target_url: {target_url}')
        yield SeleniumRequest(url=target_url,
                              headers=headers,
                              callback=self.parse,
                              wait_time=15,
                              errback=self.error_handle)

    def error_handle(self, error):
        print(error)
        self.runner.timeout = True
    

    def parse(self, response, **kwargs):
        try:
            # Parse the main category page and extract links to individual product pages
            print(f"Starting to parse")
            

            # Get the number of total pages
            less_page = response.css('div.page_number.less_than_thirty_page.pic-flex-center.pic-flex-center-column') # modify
            
            # modify
            if less_page != []:
                over_ten_pages = response.css('li.pic-page-num-box ::attr(href)').getall()  
                if over_ten_pages != []:
                    last_pages_link = over_ten_pages[-1]
                    total_pages = last_pages_link.rsplit('page=')[-1]
                else:
                    total_pages = less_page.css('ul li a ::text').getall()[-1]
            else:
                over_ten_pages = response.css('li.pic-page-num-box ::attr(href)').getall()
                last_pages_link = over_ten_pages[-1]
                total_pages = last_pages_link.rsplit('page=')[-1]
                
            print()
            print(f'Total pages: {total_pages}')
            
            #driver = response.request.meta['driver'] # close to avoid bug
            #-------------- add to avoid bugs from jupyter-lab --------------------
            options = webdriver.ChromeOptions()
            options.add_argument("--headless")
            options.add_argument("--no-sandbox")
            options.add_argument("--single-process")
            options.add_argument("--disable-dev-shm-usage")
            options.add_argument("blink-settings=imagesEnabled=false")
            options.add_argument("--remote-debugging-port=9222")
            options.add_argument("--no-zygote")
            options.add_argument("--disable-gpu")
            options.add_argument("--disable-dev-tools")
    
            driver = webdriver.Chrome(options=options)
            driver.get(response.url)
            #----------------------------------------------------------------------
            tier = {} # modify
            threads = []
            for i in range(1, int(total_pages) + 1):
                # For each page, extract product links
                if i == 1:
                    tier_content_links = response.css("div.pic-pds-infobox h2 a::attr(href)").getall() # modify
                    self.set_all_tier(self, tier)
                    self.construct_s3_tier_folder(tier)

                else:  # If the number of page is greater than 1 then
                    # For subsequent pages, click on the page number to load new content and extract links
                    
                    # modify
                    WebDriverWait(driver, 15).until(ec.element_to_be_clickable((By.XPATH, f'//li[@class="admweb-v2-page-num-box"]/a[text()="{i}"]'))).click()

                    # Get the updated page source and extract product links
                    updated_content = driver.page_source
                    updated_response = scrapy.http.HtmlResponse(url=driver.current_url, body=updated_content,
                                                                encoding='utf-8')
                    
                    tier_content_links = updated_response.css("div.pic-pds-infobox h2 a::attr(href)").getall() # modify

                # Start the threading to push the product links to SQS and/or S3
                self.start_to_push(tier_content_links, tier, threads)

            # Wait for all threads to finish before proceeding
            for thread in threads:
                thread.join()
        except Exception as e:
            print(str(e))
    
    
    
    @staticmethod
    def set_all_tier(self, tier: dict): # modify on 08/30
        tier['tier1'] = self.runner.tier1
        tier['tier2'] = self.runner.tier2
        tier['tier3'] = self.runner.tier3
        tier['tier4'] = self.runner.tier4 if self.runner.category_type == 'tier4' else ''
        tier['tier5'] = self.runner.tier5 if self.runner.category_type == 'tier5' else ''
        tier['category_type'] = self.runner.category_type

    @staticmethod
    def construct_s3_tier_folder(tier: dict):
        # Construct the base folder path
        base_path = "content"

        # Create the base folder if it doesn't exist
        # if base_path:
            # s3.put_object(Bucket=s3_crawler_content_folder, Key=f"{base_path}/")

        # Construct the folder structure based on the tiers
        for key, value in tier.items():
            if value != "" and key != 'category_type':
                folder_name = value.strip('/')
                folder_path = f"{base_path}/{folder_name}"
                print(folder_path)
            # Create the folder if it doesn't exist
            # s3.put_object(Bucket=s3_crawler_content_folder, Key=f"{folder_path}/")

            # Set the current folder path as the base path for the next iteration
                base_path = folder_path

    def start_to_push(self, tier_content_links, tier, threads):
        # For each product link, create a message and push it to SQS and/or S3
        for tier_content_link in tier_content_links:
            hash_id = hash_function(tier_content_link)
            message = {'Id': hash_id, 'MessageGroupId': hash_id,
                       'MessageBody': json.dumps({'tier_content_link': tier_content_link, 
                                                  'tier': tier}, ensure_ascii=False)}

#             thread_sqs = threading.Thread(target=self.send_to_sqs, args=(message,))
#             threads.append(thread_sqs)
#             thread_sqs.start()

#             # self.send_to_sqs(message)

#             if export_to_s3 is not None and export_to_s3 == "On":
#                 message_body = {'hash_id': hash_id, 'page_url': tier_content_link}
#                 self.send_to_s3(message_body, hash_id)
#                 thread_s3 = threading.Thread(target=self.send_to_sqs, args=(message,))
#                 threads.append(thread_s3)
#                 thread_s3.start()            
            
            

#     @staticmethod
#     def send_to_sqs(message):
#         # Send the message to SQS
#         try:
#             # print(message['MessageGroupId'])
#             response = sqs.send_message(QueueUrl=queue_tier_content_links,
#                                         MessageGroupId=message['MessageGroupId'],
#                                         MessageBody=message['MessageBody'],
#                                         MessageDeduplicationId=message['MessageGroupId'])
#             if response["ResponseMetadata"]["HTTPStatusCode"] != 200:
#                 print('Fail')
#         except Exception as e:
#             print(str(e))

#     @staticmethod
#     def send_to_s3(m, hash_id):
#         # Send the message to S3
#         try:
#             json_file_name = f'{hash_id}.json'
#             response = s3.put_object(Bucket=s3_tier_content_links, Key=json_file_name,
#                                      Body=json.dumps(m, ensure_ascii=False))
#             # check if it's successful
#             if response["ResponseMetadata"]["HTTPStatusCode"] != 200:
#                 print('Fail')
#         except Exception as e:
#             print(str(e))

    def spider_closed(self):
        # Handle the spider closing event
        self.runner.timeout = False


In [61]:
class LambdaRunner:
    target_url = ""
    receipt_handle = ""
    tier4_category_object = None
    timeout = False

    def __init__(self, url):
        self.finished = threading.Event()
        self.results = []
        if url != "":
            self.target_url = url

    def run_spider(self):
        # Create a CrawlerRunner with project settings
        settings = get_project_settings()
        runner = CrawlerRunner(settings)
        if self.target_url == "":
            self.get_tier4_url_from_sqs()

        # Callback function to handle the spider results
        def handle_results(result):
            self.results.append(result)

            # Check if the spider has finished running
            if len(self.results) == 1:
                self.finished.set()

        # Start the first spider run
        deferred = runner.crawl(IOpenMallSpider, runner=self) # modify
        deferred.addCallback(handle_results)

        # Start the reactor
        runner.join()

    def wait_for_completion(self):
        self.finished.wait()

    def get_results(self):
        return self.results

    def get_tier4_url_from_sqs(self):
        response = sqs.receive_message(
            QueueUrl=queue_tier_links,
            MaxNumberOfMessages=1,  # Retrieve 10 messages
            WaitTimeSeconds=0  # Maximum time to wait for messages (long polling)
        )
        messages = response.get('Messages', [])
        if messages is not None:
            self.tier4_category_object = messages[0]
        else:
            msg = "All consumed."
            print(msg)
            return msg

        # Delete messages from SQS
        for message in messages:
            sqs.delete_message(
                QueueUrl=queue_tier_links,
                ReceiptHandle=message['ReceiptHandle']
            )


def handler(event, context):
    try:
        # Check if the function was triggered by an HTTP request or Lambda event
        print(f"Starting to crawl: {datetime.datetime.now()}")
        times = 0
        if "statusCode" not in event:
            # If the function was not triggered by retry
            runner = LambdaRunner("")
            runner.tier1 = event["tier1_category_name"]    # add on 08/30
            runner.tier2 = event["tier2_category_name"]    # add on 08/30
            runner.tier3 = event["tier3_category_name"]    # add on 08/30
            runner.tier4 = event["tier4_category_name"] if event["category_type"] == 'tier4' else None   # add on 08/30
            runner.tier5 = event["tier5_category_name"] if event["category_type"] == 'tier5' else None   # add on 08/30
            runner.category_type = event["category_type"]  # add on 08/30
            runner.run_spider()
            runner.wait_for_completion()
            print(f"End date and time:{datetime.datetime.now()}")
        else:
            times = int(event["times"])
            if times < 4:
                runner = LambdaRunner(event["category_link"])
                runner.input_url = event["category_link"]
                runner.tier1 = event["tier1_category_name"]    # add on 08/30
                runner.tier2 = event["tier2_category_name"]    # add on 08/30
                runner.tier3 = event["tier3_category_name"]    # add on 08/30
                runner.tier4 = event["tier4_category_name"] if event["category_type"] == 'tier4' else None   # add on 08/30
                runner.tier5 = event["tier5_category_name"] if event["category_type"] == 'tier5' else None   # add on 08/30
                runner.category_type = event["category_type"]  # add on 08/30
                runner.run_spider()
                runner.wait_for_completion()
                print(f"End date and time: {datetime.datetime.now()}")
            else:
                print(f'Retry too many times, 429: {event["category_link"]}')
                # If the retry count is 4 or more, return an HTTP 429 response indicating Too Many Requests
                return {
                    'statusCode': 429,
                    'body': "Retry too many times",
                    'times': times,
                    "category_link": event["category_link"]
                }

        times = times + 1
        if not runner.timeout:
            print('success')
            # If the LambdaRunner completed successfully, return an HTTP 200 response with the completion details
            return {
                'statusCode': 200,
                'body': 'Completed!',
                'times': times,
                # "category_link": event["category_link"],
                # "category_info": runner.category_info
            }
        else:
            print('timeout')
            # If the LambdaRunner timed out, return an HTTP 408 response with the category objects
            return {
                'statusCode': 408,
                'times': times,
                "category_link": json.loads(runner.tier4_category_object['Body'])['category_link']
            }
    except Exception as e:
        print(f'fail:{e}')
        return {
            'statusCode': 500,
            'body': json.dumps({'error': str(e)})
        }


In [62]:
handler({'statusCode': 408,'times': 1, 
        'tier1_category_name': '休閒美食', 
         'tier2_category_name': '美食、伴手禮', 
         'tier3_category_name': '乳製品、蛋', 
         'tier4_category_name': '其他', 
         'tier5_category_name': '', 
         'category_link': 'https://mall.iopenmall.tw/iopen/index.php?action=store_product_sort&prod_sort_ou=456', 
         'category_type': 'tier4'}, "")



Starting to crawl: 2023-08-30 07:42:44.499055
Starting start_requests
target_url: https://mall.iopenmall.tw/iopen/index.php?action=store_product_sort&prod_sort_ou=456
Starting to parse

Total pages: 1
content/休閒美食
content/休閒美食/美食、伴手禮
content/休閒美食/美食、伴手禮/乳製品、蛋
content/休閒美食/美食、伴手禮/乳製品、蛋/其他
End date and time: 2023-08-30 07:42:47.840058
success


{'statusCode': 200, 'body': 'Completed!', 'times': 2}